In [104]:
# lets now import all the required libraries.
import selenium
import pandas as pd
import time
import warnings   
from bs4 import BeautifulSoup
warnings.filterwarnings("ignore")

# Importing selenium webdriver 
from selenium import webdriver

# Importing required Exceptions which needs to handled
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException,ElementNotInteractableException

#Importing requests
import requests

# importing regex
import re

In [105]:
# Activating the chrome browser
driver = webdriver.Chrome(r"C:\Web Driver\chromedriver.exe")
time.sleep(2)

# Opening the CarDekho web url
url = "https://www.cleartrip.com/"
driver.get(url)
time.sleep(2)

### From New Delhi to Chennai

In [106]:
# Click on from textbox.
from_location = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div/div[2]/div/div[3]/div[1]/div[1]/div/div/div/input")
from_location.click()
time.sleep(2)

# Then click the search bar and enter city name.
from_location_city = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div/div[2]/div/div[3]/div[1]/div[1]/div/div/div/input")
from_location_city.send_keys('New Delhi')
time.sleep(4)

# Then click on suggestion name of searched city.
suggrstion_city = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div/div[2]/div/div[3]/div[1]/div[1]/div/div/div[2]/ul/li/p")
suggrstion_city.click()
time.sleep(2)

In [107]:
# Click on to textbox.
to_location = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div/div[2]/div/div[3]/div[1]/div[5]/div/div/div/input")
to_location.click()
time.sleep(2)

# Then click the search bar and enter city name.
to_location_city = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div/div[2]/div/div[3]/div[1]/div[5]/div/div/div/input")
to_location_city.send_keys('Chennai')
time.sleep(4)

# Then click on suggestion name of searched city.
suggrstion_city = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div/div[2]/div/div[3]/div[1]/div[5]/div/div/div[2]/ul/li/p")
suggrstion_city.click()
time.sleep(2)

In [108]:
# Click on Depart on dropdown.
dep_date = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div/div[2]/div/div[3]/div[3]/div/div/div/div/button")
dep_date.click()
time.sleep(2)

# Click on depart date.
depart_date = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div/div[2]/div/div[3]/div[3]/div/div/div/div/div/ul/div[2]/div/div[2]/div[1]/div[3]/div[2]/div[4]/div/div")
depart_date.click()
time.sleep(2)

In [109]:
# Click on search button.
search_btn = driver.find_element_by_xpath("/html/body/div[1]/div/div/div[2]/div/div[2]/div/div[7]/div[2]/button")
search_btn.click()
time.sleep(4)

In [110]:
# Scrap details for airline name, dep. time, arri. time, duration, total stops and price.
for _ in range(20):
    driver.execute_script("window.scrollBy(0,1000)")
    time.sleep(2)
    
    #Scrap airline name
    airline_name = []
    try:                    
        airline_name_tags=driver.find_elements_by_xpath("//div[@class='flex flex-row nmx-1']/div[3]/p[1]")
        for i in airline_name_tags:
            airline_name.append(i.text)
    except NoSuchElementException:#handling no such element exception
        airline_name.append('No details available')
        
    #Scrap departure time
    departure_time = []
    try:                    
        departure_time_tags=driver.find_elements_by_xpath("//div[@class='ow-tuple-container__details-a__time ms-grid-row-2']/div[1]/p")
        for i in departure_time_tags:
            departure_time.append(i.text)
    except NoSuchElementException:#handling no such element exception
        departure_time.append('No details available')
        
    #Scrap arrival time
    arrival_time = []
    try:                    
        arrival_time_tags=driver.find_elements_by_xpath("//div[@class='ow-tuple-container__details-a__time ms-grid-row-2']/div[3]/p")
        for i in arrival_time_tags:
            arrival_time.append(i.text.split('\n')[0])
    except NoSuchElementException:#handling no such element exception
        arrival_time.append('No details available')
        
    #Scrap duration
    duration = []
    try:                    
        duration_tags=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-1']/p[1]")
        for i in duration_tags:
            duration.append(i.text)
    except NoSuchElementException:#handling no such element exception
        duration.append('No details available')
        
    #Scrap total_stops
    flight_stop = []
    try:                    
        flight_stop_tags=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-2']/p")
        for i in flight_stop_tags:
            flight_stop.append(i.text)
    except NoSuchElementException:#handling no such element exception
        flight_stop.append('No details available')
        
    #Scrap price
    price = []
    try:
        price_tags=driver.find_elements_by_xpath("//div[@class='flex flex-column flex-center flex-right p-relative']")
        for i in price_tags:
            if('\n' in i.text):
                price.append(i.text.split('\n')[1].replace('₹','').replace(',',''))
            else:
                price.append(i.text.replace('₹','').replace(',',''))
                        
    except NoSuchElementException:#handling no such element exception
        price.append('No details available')

In [111]:
# Fetching url to open each product.
flight_details_url = []
for _ in range(20):
    driver.execute_script("window.scrollBy(0,1000)")
    time.sleep(1)
    
url = driver.find_elements_by_xpath('//span[@class="c-pointer c-secondary-500 hover:td-underline fs-2"]')
for i in url:
    try:
        flight_details_url.append(i.get_attribute('href'))
    except NoSuchElementException:
        flight_details_url.append('-')

In [112]:
len(flight_details_url)

69

In [113]:
# Scrap details for date of journey, source, destination.
date_of_journey = []
source = []
destination = []

for i in flight_details_url:
    #time.sleep(1) 
    
    #scarping date of journey
    try:                    
        doj_tags=driver.find_element_by_xpath("//div[@class='br-4 bw-1 bc-neutral-100 o-hidden ba mb-2']/div[2]/div[1]/div/div[2]/div[2]/span")
        date_of_journey.append(doj_tags.text.split(',')[1].strip().replace('Aug','08').replace(' ','/'))
    except NoSuchElementException:#handling no such element exception
        date_of_journey.append('No details available')
    
    #scarping flight source
    try:                    
        source_tags=driver.find_element_by_xpath("//h3[@class='fs-3 c-neutral-900 fw-500']")
        source.append(source_tags.text.split('→')[0])
    except NoSuchElementException:#handling no such element exception
        source.append('No details available') 
    
    #scarping flight destination
    try:                    
        destination_tags=driver.find_element_by_xpath("//h3[@class='fs-3 c-neutral-900 fw-500']")
        destination.append(destination_tags.text.split('→')[1])
    except NoSuchElementException:#handling no such element exception
        destination.append('No details available')

In [114]:
print(len(airline_name),len(date_of_journey),len(source),len(destination),len(departure_time),len(arrival_time),len(duration),len(flight_stop),len(price))

69 69 69 69 69 69 69 69 69


In [115]:
# Create dataframe.
import pandas as pd
df = pd.DataFrame({
    "Airline_name" : airline_name,
    "Date_of_journey" : date_of_journey,
    "Source" : source,
    "Destination" : destination,
    "Departure_time" : departure_time,
    "Arrival_time" : arrival_time,
    "Duration" : duration,
    "Total_stops" : flight_stop,
    "Price" : price,
})
df

,Airline_name,Date_of_journey,Source,Destination,Departure_time,Arrival_time,Duration,Total_stops,Price
0,Air Asia,11/08/2022,New Delhi,Chennai,12:40,22:00,9h 20m,1 stop,7949
1,Vistara,11/08/2022,New Delhi,Chennai,19:55,22:45,2h 50m,non-stop,9418
2,IndiGo,11/08/2022,New Delhi,Chennai,07:10,10:00,2h 50m,non-stop,9419
3,IndiGo,11/08/2022,New Delhi,Chennai,08:30,11:20,2h 50m,non-stop,9419
4,IndiGo,11/08/2022,New Delhi,Chennai,17:25,20:15,2h 50m,non-stop,9419
...,...,...,...,...,...,...,...,...,...
64,SpiceJet,11/08/2022,New Delhi,Chennai,21:40,21:30,23h 50m,1 stop,19388
65,SpiceJet,11/08/2022,New Delhi,Chennai,20:00,21:30,25h 30m,1 stop,19388
66,SpiceJet,11/08/2022,New Delhi,Chennai,19:00,21:30,26h 30m,1 stop,19388
67,Vistara,11/08/2022,New Delhi,Chennai,11:40,19:50,8h 10m,1 stop,19918


### From Mumbai to Bangalore

In [15]:
# Click on from textbox.
from_location = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div/input")
from_location.click()
time.sleep(2)

# Then click the search bar and enter city name.
from_location_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div/input")
from_location_city.send_keys('Mumbai')
time.sleep(4)

# Then click on suggestion name of searched city.
suggrstion_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div[2]/ul/li/p")
suggrstion_city.click()
time.sleep(2)

In [16]:
# Click on to textbox.
to_location = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[1]/input")
to_location.click()
time.sleep(2)

# Then click the search bar and enter city name.
to_location_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[1]/input")
to_location_city.send_keys('Bangalore')
time.sleep(4)

# Then click on suggestion name of searched city.
suggrstion_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[2]/ul/li[1]/p")
suggrstion_city.click()
time.sleep(2)

In [17]:
# Click on Depart on dropdown.
dep_date = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[8]/div/button[1]")
dep_date.click()
time.sleep(2)

# Click on depart date.
depart_date = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[8]/div/div[2]/ul/div[3]/div/div[2]/div[2]/div[3]/div[2]/div[3]/div/div")
depart_date.click()
time.sleep(2)

In [18]:
# Click on search button.
search_btn = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[12]/button")
search_btn.click()
time.sleep(4)

In [19]:
# Scrap details for airline name, dep. time, arri. time, duration, total stops and price.
for _ in range(20):
    driver.execute_script("window.scrollBy(0,2000)")
    time.sleep(1)
    
    #Scrap airline name
    airline_name = []
    try:                    
        airline_name_tags=driver.find_elements_by_xpath("//div[@class='flex flex-row nmx-1']/div[3]/p[1]")
        for i in airline_name_tags:
            airline_name.append(i.text)
    except NoSuchElementException:#handling no such element exception
        airline_name.append('No details available')
        
    #Scrap departure time
    departure_time = []
    try:                    
        departure_time_tags=driver.find_elements_by_xpath("//div[@class='ow-tuple-container__details-a__time ms-grid-row-2']/div[1]/p")
        for i in departure_time_tags:
            departure_time.append(i.text)
    except NoSuchElementException:#handling no such element exception
        departure_time.append('No details available')
        
    #Scrap arrival time
    arrival_time = []
    try:                    
        arrival_time_tags=driver.find_elements_by_xpath("//div[@class='ow-tuple-container__details-a__time ms-grid-row-2']/div[3]/p")
        for i in arrival_time_tags:
            arrival_time.append(i.text.split('\n')[0])
    except NoSuchElementException:#handling no such element exception
        arrival_time.append('No details available')
        
    #Scrap duration
    duration = []
    try:                    
        duration_tags=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-1']/p[1]")
        for i in duration_tags:
            duration.append(i.text)
    except NoSuchElementException:#handling no such element exception
        duration.append('No details available')
        
    #Scrap total_stops
    flight_stop = []
    try:                    
        flight_stop_tags=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-2']/p")
        for i in flight_stop_tags:
            flight_stop.append(i.text)
    except NoSuchElementException:#handling no such element exception
        flight_stop.append('No details available')
        
    #Scrap price
    price = []
    try:
        price_tags=driver.find_elements_by_xpath("//div[@class='flex flex-column flex-center flex-right p-relative']")
        for i in price_tags:
            if('\n' in i.text):
                price.append(i.text.split('\n')[1].replace('₹','').replace(',',''))
            else:
                price.append(i.text.replace('₹','').replace(',',''))
                        
    except NoSuchElementException:#handling no such element exception
        price.append('No details available')

In [20]:
# Fetching url to open each product.
flight_details_url = []
for _ in range(20):
    driver.execute_script("window.scrollBy(0,2000)")
    time.sleep(1)
    
url = driver.find_elements_by_xpath('//span[@class="c-pointer c-secondary-500 hover:td-underline fs-2"]')
for i in url:
    try:
        flight_details_url.append(i.get_attribute('href'))
    except NoSuchElementException:
        flight_details_url.append('-')

In [21]:
len(flight_details_url)

197

In [22]:
# Scrap details for date of journey, source, destination.
date_of_journey = []
source = []
destination = []

for i in flight_details_url:
    #time.sleep(1) 
    
    #scarping date of journey
    try:                    
        doj_tags=driver.find_element_by_xpath("//div[@class='br-4 bw-1 bc-neutral-100 o-hidden ba mb-2']/div[2]/div[1]/div/div[2]/div[2]/span")
        date_of_journey.append(doj_tags.text.split(',')[1].strip().replace('Sep','09').replace(' ','/'))
    except NoSuchElementException:#handling no such element exception
        date_of_journey.append('No details available')
    
    #scarping flight source
    try:                    
        source_tags=driver.find_element_by_xpath("//h3[@class='fs-3 c-neutral-900 fw-500']")
        source.append(source_tags.text.split('→')[0])
    except NoSuchElementException:#handling no such element exception
        source.append('No details available') 
    
    #scarping flight destination
    try:                    
        destination_tags=driver.find_element_by_xpath("//h3[@class='fs-3 c-neutral-900 fw-500']")
        destination.append(destination_tags.text.split('→')[1])
    except NoSuchElementException:#handling no such element exception
        destination.append('No details available')

In [23]:
print(len(airline_name),len(date_of_journey),len(source),len(destination),len(departure_time),len(arrival_time),len(duration),len(flight_stop),len(price))

197 197 197 197 197 197 197 197 197


In [24]:
#Create dataframe.
import pandas as pd
df2 = pd.DataFrame({
    "Airline_name" : airline_name,
    "Date_of_journey" : date_of_journey,
    "Source" : source,
    "Destination" : destination,
    "Departure_time" : departure_time,
    "Arrival_time" : arrival_time,
    "Duration" : duration,
    "Total_stops" : flight_stop,
    "Price" : price
})
df2

,Airline_name,Date_of_journey,Source,Destination,Departure_time,Arrival_time,Duration,Total_stops,Price
0,GO FIRST,7/09/2022,Mumbai,Bangalore,05:50,07:35,1h 45m,non-stop,3469
1,GO FIRST,7/09/2022,Mumbai,Bangalore,20:30,22:15,1h 45m,non-stop,3469
2,GO FIRST,7/09/2022,Mumbai,Bangalore,16:30,18:20,1h 50m,non-stop,3469
3,Air Asia,7/09/2022,Mumbai,Bangalore,13:05,14:45,1h 40m,non-stop,3472
4,Air Asia,7/09/2022,Mumbai,Bangalore,19:00,20:50,1h 50m,non-stop,3472
...,...,...,...,...,...,...,...,...,...
192,Vistara,7/09/2022,Mumbai,Bangalore,17:35,23:20,5h 45m,1 stop,11427
193,Vistara,7/09/2022,Mumbai,Bangalore,06:00,11:50,5h 50m,1 stop,11427
194,Vistara,7/09/2022,Mumbai,Bangalore,12:25,18:50,6h 25m,1 stop,11427
195,Vistara,7/09/2022,Mumbai,Bangalore,10:25,17:05,6h 40m,1 stop,11427


### From Ahmedabad to Hyderabad

In [27]:
# Click on from textbox.
from_location = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div/input")
from_location.click()
time.sleep(2)

# Then click the search bar and enter city name.
from_location_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div/input")
from_location_city.send_keys('Ahmedabad')
time.sleep(4)

# Then click on suggestion name of searched city.
suggrstion_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div[2]/ul/li/p")
suggrstion_city.click()
time.sleep(2)

In [36]:
# Click on to textbox.
to_location = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[1]/input")
to_location.click()
time.sleep(2)

# Then click the search bar and enter city name.
to_location_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[1]/input")
to_location_city.send_keys('Hyderabad')
time.sleep(4)

# Then click on suggestion name of searched city.
suggrstion_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[2]/ul/li[1]/p")
suggrstion_city.click()
time.sleep(2)

In [32]:
# Click on Depart on dropdown.
dep_date = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[8]/div/button[1]")
dep_date.click()
time.sleep(2)

In [33]:
# Click on depart date.
depart_date = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[8]/div/div[2]/ul/div[3]/div/div[2]/div[1]/div[3]/div[1]/div[7]/div/div")
depart_date.click()
time.sleep(2)

In [37]:
# Click on search button.
search_btn = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[12]/button")
search_btn.click()
time.sleep(4)

In [39]:
# Scrap details for airline name, dep. time, arri. time, duration, total stops and price.
for _ in range(20):
    driver.execute_script("window.scrollBy(0,100)")
    time.sleep(1)
    
    #Scrap airline name
    airline_name = []
    try:                    
        airline_name_tags=driver.find_elements_by_xpath("//div[@class='flex flex-row nmx-1']/div[3]/p[1]")
        for i in airline_name_tags:
            airline_name.append(i.text)
    except NoSuchElementException:#handling no such element exception
        airline_name.append('No details available')
        
    #Scrap departure time
    departure_time = []
    try:                    
        departure_time_tags=driver.find_elements_by_xpath("//div[@class='ow-tuple-container__details-a__time ms-grid-row-2']/div[1]/p")
        for i in departure_time_tags:
            departure_time.append(i.text)
    except NoSuchElementException:#handling no such element exception
        departure_time.append('No details available')
        
    #Scrap arrival time
    arrival_time = []
    try:                    
        arrival_time_tags=driver.find_elements_by_xpath("//div[@class='ow-tuple-container__details-a__time ms-grid-row-2']/div[3]/p")
        for i in arrival_time_tags:
            arrival_time.append(i.text.split('\n')[0])
    except NoSuchElementException:#handling no such element exception
        arrival_time.append('No details available')
        
    #Scrap duration
    duration = []
    try:                    
        duration_tags=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-1']/p[1]")
        for i in duration_tags:
            duration.append(i.text)
    except NoSuchElementException:#handling no such element exception
        duration.append('No details available')
        
    #Scrap total_stops
    flight_stop = []
    try:                    
        flight_stop_tags=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-2']/p")
        for i in flight_stop_tags:
            flight_stop.append(i.text)
    except NoSuchElementException:#handling no such element exception
        flight_stop.append('No details available')
        
    #Scrap price
    price = []
    try:
        price_tags=driver.find_elements_by_xpath("//div[@class='flex flex-column flex-center flex-right p-relative']")
        for i in price_tags:
            if('\n' in i.text):
                price.append(i.text.split('\n')[1].replace('₹','').replace(',',''))
            else:
                price.append(i.text.replace('₹','').replace(',',''))
                        
    except NoSuchElementException:#handling no such element exception
        price.append('No details available')

In [40]:
# Fetching url to open each product.
flight_details_url = []
for _ in range(20):
    driver.execute_script("window.scrollBy(0,100)")
    time.sleep(1)
    
url = driver.find_elements_by_xpath('//span[@class="c-pointer c-secondary-500 hover:td-underline fs-2"]')
for i in url:
    try:
        flight_details_url.append(i.get_attribute('href'))
    except NoSuchElementException:
        flight_details_url.append('-')

In [41]:
len(flight_details_url)

13

In [43]:
# Scrap details for date of journey, source, destination.
date_of_journey = []
source = []
destination = []

for i in flight_details_url:
    #time.sleep(1) 
    
    #scarping date of journey
    try:                    
        doj_tags=driver.find_element_by_xpath("//div[@class='br-4 bw-1 bc-neutral-100 o-hidden ba mb-2']/div[2]/div[1]/div/div[2]/div[2]/span")
        date_of_journey.append(doj_tags.text.split(',')[1].strip().replace('Oct','10').replace(' ','/'))
    except NoSuchElementException:#handling no such element exception
        date_of_journey.append('No details available')
    
    #scarping flight source
    try:                    
        source_tags=driver.find_element_by_xpath("//h3[@class='fs-3 c-neutral-900 fw-500']")
        source.append(source_tags.text.split('→')[0])
    except NoSuchElementException:#handling no such element exception
        source.append('No details available') 
    
    #scarping flight destination
    try:                    
        destination_tags=driver.find_element_by_xpath("//h3[@class='fs-3 c-neutral-900 fw-500']")
        destination.append(destination_tags.text.split('→')[1])
    except NoSuchElementException:#handling no such element exception
        destination.append('No details available')

In [44]:
print(len(airline_name),len(date_of_journey),len(source),len(destination),len(departure_time),len(arrival_time),len(duration),len(flight_stop),len(price))

13 13 13 13 13 13 13 13 13


In [45]:
#Create dataframe.
import pandas as pd
df3 = pd.DataFrame({
    "Airline_name" : airline_name,
    "Date_of_journey" : date_of_journey,
    "Source" : source,
    "Destination" : destination,
    "Departure_time" : departure_time,
    "Arrival_time" : arrival_time,
    "Duration" : duration,
    "Total_stops" : flight_stop,
    "Price" : price,
})
df3

,Airline_name,Date_of_journey,Source,Destination,Departure_time,Arrival_time,Duration,Total_stops,Price
0,IndiGo,2/10/2022,Ahmedabad,Hyderabad,08:50,10:35,1h 45m,non-stop,5586
1,IndiGo,2/10/2022,Ahmedabad,Hyderabad,15:50,17:35,1h 45m,non-stop,5586
2,IndiGo,2/10/2022,Ahmedabad,Hyderabad,20:50,22:35,1h 45m,non-stop,5586
3,IndiGo,2/10/2022,Ahmedabad,Hyderabad,05:45,10:35,4h 50m,1 stop,5849
4,IndiGo,2/10/2022,Ahmedabad,Hyderabad,22:55,00:30,1h 35m,non-stop,6006
5,IndiGo,2/10/2022,Ahmedabad,Hyderabad,04:25,08:45,4h 20m,1 stop,6407
6,IndiGo,2/10/2022,Ahmedabad,Hyderabad,03:15,07:35,4h 20m,1 stop,8369
7,IndiGo,2/10/2022,Ahmedabad,Hyderabad,20:50,01:20,4h 30m,1 stop,8369
8,IndiGo,2/10/2022,Ahmedabad,Hyderabad,22:55,03:20,4h 25m,1 stop,9261
9,IndiGo,2/10/2022,Ahmedabad,Hyderabad,04:30,10:05,5h 35m,1 stop,9734


In [130]:
 df.to_excel(r'flight_data.xlsx')

In [48]:
df

,Airline_name,Date_of_journey,Source,Destination,Departure_time,Arrival_time,Duration,Total_stops,Price
0,GO FIRST,11/07/2022,New Delhi,Chennai,20:50,07:45,10h 55m,1 stop,7949
1,GO FIRST,11/07/2022,New Delhi,Chennai,18:20,07:45,13h 25m,1 stop,7949
2,Vistara,11/07/2022,New Delhi,Chennai,13:35,16:25,2h 50m,non-stop,9418
3,IndiGo,11/07/2022,New Delhi,Chennai,07:10,10:00,2h 50m,non-stop,9419
4,IndiGo,11/07/2022,New Delhi,Chennai,08:30,11:20,2h 50m,non-stop,9419
...,...,...,...,...,...,...,...,...,...
69,SpiceJet,11/07/2022,New Delhi,Chennai,21:40,21:30,23h 50m,1 stop,18563
70,SpiceJet,11/07/2022,New Delhi,Chennai,20:00,21:30,25h 30m,1 stop,18563
71,SpiceJet,11/07/2022,New Delhi,Chennai,19:00,21:30,26h 30m,1 stop,18563
72,Vistara,11/07/2022,New Delhi,Chennai,14:20,19:50,5h 30m,1 stop,19183


### From Mumbai to Surat 

In [49]:
# Click on from textbox.
from_location = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div/input")
from_location.click()
time.sleep(2)

# Then click the search bar and enter city name.
from_location_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div/input")
from_location_city.send_keys('Mumbai')
time.sleep(4)

# Then click on suggestion name of searched city.
suggrstion_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div[2]/ul/li/p")
suggrstion_city.click()
time.sleep(2)

In [50]:
# Click on to textbox.
to_location = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[1]/input")
to_location.click()
time.sleep(2)

# Then click the search bar and enter city name.
to_location_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[1]/input")
to_location_city.send_keys('Surat')
time.sleep(4)

# Then click on suggestion name of searched city.
suggrstion_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[2]/ul/li[1]/p")
suggrstion_city.click()
time.sleep(2)

In [53]:
# Click on Depart on dropdown.
dep_date = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[8]/div/button[1]")
dep_date.click()
time.sleep(2)

# Click on depart date.
depart_date = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[8]/div/div[2]/ul/div[3]/div/div[2]/div[1]/div[3]/div[4]/div[2]/div/div")
depart_date.click()
time.sleep(2)

In [54]:
# Click on search button.
search_btn = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[12]/button")
search_btn.click()
time.sleep(4)

In [55]:
# Scrap details for airline name, dep. time, arri. time, duration, total stops and price.
for _ in range(20):
    driver.execute_script("window.scrollBy(0,100)")
    time.sleep(1)
    
    #Scrap airline name
    airline_name = []
    try:                    
        airline_name_tags=driver.find_elements_by_xpath("//div[@class='flex flex-row nmx-1']/div[3]/p[1]")
        for i in airline_name_tags:
            airline_name.append(i.text)
    except NoSuchElementException:#handling no such element exception
        airline_name.append('No details available')
        
    #Scrap departure time
    departure_time = []
    try:                    
        departure_time_tags=driver.find_elements_by_xpath("//div[@class='ow-tuple-container__details-a__time ms-grid-row-2']/div[1]/p")
        for i in departure_time_tags:
            departure_time.append(i.text)
    except NoSuchElementException:#handling no such element exception
        departure_time.append('No details available')
        
    #Scrap arrival time
    arrival_time = []
    try:                    
        arrival_time_tags=driver.find_elements_by_xpath("//div[@class='ow-tuple-container__details-a__time ms-grid-row-2']/div[3]/p")
        for i in arrival_time_tags:
            arrival_time.append(i.text.split('\n')[0])
    except NoSuchElementException:#handling no such element exception
        arrival_time.append('No details available')
        
    #Scrap duration
    duration = []
    try:                    
        duration_tags=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-1']/p[1]")
        for i in duration_tags:
            duration.append(i.text)
    except NoSuchElementException:#handling no such element exception
        duration.append('No details available')
        
    #Scrap total_stops
    flight_stop = []
    try:                    
        flight_stop_tags=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-2']/p")
        for i in flight_stop_tags:
            flight_stop.append(i.text)
    except NoSuchElementException:#handling no such element exception
        flight_stop.append('No details available')
        
    #Scrap price
    price = []
    try:
        price_tags=driver.find_elements_by_xpath("//div[@class='flex flex-column flex-center flex-right p-relative']")
        for i in price_tags:
            if('\n' in i.text):
                price.append(i.text.split('\n')[1].replace('₹','').replace(',',''))
            else:
                price.append(i.text.replace('₹','').replace(',',''))
                        
    except NoSuchElementException:#handling no such element exception
        price.append('No details available')

In [56]:
# Fetching url to open each product.
flight_details_url = []
for _ in range(20):
    driver.execute_script("window.scrollBy(0,100)")
    time.sleep(1)
    
url = driver.find_elements_by_xpath('//span[@class="c-pointer c-secondary-500 hover:td-underline fs-2"]')
for i in url:
    try:
        flight_details_url.append(i.get_attribute('href'))
    except NoSuchElementException:
        flight_details_url.append('-')

In [57]:
len(flight_details_url)

12

In [61]:
# Scrap details for date of journey, source, destination.
date_of_journey = []
source = []
destination = []

for i in flight_details_url:
    #time.sleep(1) 
    
    #scarping date of journey
    try:                    
        doj_tags=driver.find_element_by_xpath("//div[@class='br-4 bw-1 bc-neutral-100 o-hidden ba mb-2']/div[2]/div[1]/div/div[2]/div[2]/span")
        date_of_journey.append(doj_tags.text.split(',')[1].strip().replace('Sep','09').replace(' ','/'))
    except NoSuchElementException:#handling no such element exception
        date_of_journey.append('No details available')
    
    #scarping flight source
    try:                    
        source_tags=driver.find_element_by_xpath("//h3[@class='fs-3 c-neutral-900 fw-500']")
        source.append(source_tags.text.split('→')[0])
    except NoSuchElementException:#handling no such element exception
        source.append('No details available') 
    
    #scarping flight destination
    try:                    
        destination_tags=driver.find_element_by_xpath("//h3[@class='fs-3 c-neutral-900 fw-500']")
        destination.append(destination_tags.text.split('→')[1])
    except NoSuchElementException:#handling no such element exception
        destination.append('No details available')

In [62]:
print(len(airline_name),len(date_of_journey),len(source),len(destination),len(departure_time),len(arrival_time),len(duration),len(flight_stop),len(price))

12 12 12 12 12 12 12 12 12


In [63]:
#Create dataframe.
import pandas as pd
df4 = pd.DataFrame({
    "Airline_name" : airline_name,
    "Date_of_journey" : date_of_journey,
    "Source" : source,
    "Destination" : destination,
    "Departure_time" : departure_time,
    "Arrival_time" : arrival_time,
    "Duration" : duration,
    "Total_stops" : flight_stop,
    "Price" : price,
})
df4

,Airline_name,Date_of_journey,Source,Destination,Departure_time,Arrival_time,Duration,Total_stops,Price
0,Air India,20/09/2022,Mumbai,Surat,13:00,20:45,7h 45m,1 stop,10668
1,Air India,20/09/2022,Mumbai,Surat,10:00,20:45,10h 45m,1 stop,10668
2,Air India,20/09/2022,Mumbai,Surat,09:00,20:45,11h 45m,1 stop,10668
3,Air India,20/09/2022,Mumbai,Surat,08:00,20:45,12h 45m,1 stop,10668
4,Air India,20/09/2022,Mumbai,Surat,07:00,20:45,13h 45m,1 stop,10668
5,Air India,20/09/2022,Mumbai,Surat,21:00,20:45,23h 45m,1 stop,10668
6,Air India,20/09/2022,Mumbai,Surat,19:00,20:45,25h 45m,1 stop,10668
7,Air India,20/09/2022,Mumbai,Surat,18:00,20:45,26h 45m,1 stop,10668
8,Air India,20/09/2022,Mumbai,Surat,17:00,20:45,27h 45m,1 stop,10668
9,Air India,20/09/2022,Mumbai,Surat,07:20,20:45,13h 25m,2 stops,11193


### From Pune to Bangalore

In [81]:
# Click on from textbox.
from_location = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div/input")
from_location.click()
time.sleep(2)

# Then click the search bar and enter city name.
from_location_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div/input")
from_location_city.send_keys('Pune')
time.sleep(4)

# Then click on suggestion name of searched city.
suggrstion_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[3]/div/div[2]/ul/li/p")
suggrstion_city.click()
time.sleep(2)

In [82]:
# Click on to textbox.
to_location = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[1]/input")
to_location.click()
time.sleep(2)

# Then click the search bar and enter city name.
to_location_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[1]/input")
to_location_city.send_keys('Bangalore')
time.sleep(4)

# Then click on suggestion name of searched city.
suggrstion_city = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[6]/div/div[2]/ul/li/p")
suggrstion_city.click()
time.sleep(2)

In [83]:
# Click on Depart on dropdown.
dep_date = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[8]/div/button[1]")
dep_date.click()
time.sleep(2)

In [84]:
# Click on depart date.
depart_date = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[8]/div/div[2]/ul/div[3]/div/div[2]/div[2]/div[3]/div[2]/div[4]/div/div")
depart_date.click()
time.sleep(2)

# Click on search button.
search_btn = driver.find_element_by_xpath("/html/body/div[1]/div/header/div[2]/div/div/div/div/div[12]/button")
search_btn.click()
time.sleep(4)

In [87]:
# Scrap details for airline name, dep. time, arri. time, duration, total stops and price.
for _ in range(20):
    driver.execute_script("window.scrollBy(0,300)")
    time.sleep(1)
    
    #Scrap airline name
    airline_name = []
    try:                    
        airline_name_tags=driver.find_elements_by_xpath("//div[@class='flex flex-row nmx-1']/div[3]/p[1]")
        for i in airline_name_tags:
            airline_name.append(i.text)
    except NoSuchElementException:#handling no such element exception
        airline_name.append('No details available')
        
    #Scrap departure time
    departure_time = []
    try:                    
        departure_time_tags=driver.find_elements_by_xpath("//div[@class='ow-tuple-container__details-a__time ms-grid-row-2']/div[1]/p")
        for i in departure_time_tags:
            departure_time.append(i.text)
    except NoSuchElementException:#handling no such element exception
        departure_time.append('No details available')
        
    #Scrap arrival time
    arrival_time = []
    try:                    
        arrival_time_tags=driver.find_elements_by_xpath("//div[@class='ow-tuple-container__details-a__time ms-grid-row-2']/div[3]/p")
        for i in arrival_time_tags:
            arrival_time.append(i.text.split('\n')[0])
    except NoSuchElementException:#handling no such element exception
        arrival_time.append('No details available')
        
    #Scrap duration
    duration = []
    try:                    
        duration_tags=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-1']/p[1]")
        for i in duration_tags:
            duration.append(i.text)
    except NoSuchElementException:#handling no such element exception
        duration.append('No details available')
        
    #Scrap total_stops
    flight_stop = []
    try:                    
        flight_stop_tags=driver.find_elements_by_xpath("//div[@class='ms-grid-column-3 ms-grid-row-2']/p")
        for i in flight_stop_tags:
            flight_stop.append(i.text)
    except NoSuchElementException:#handling no such element exception
        flight_stop.append('No details available')
        
    #Scrap price
    price = []
    try:
        price_tags=driver.find_elements_by_xpath("//div[@class='flex flex-column flex-center flex-right p-relative']")
        for i in price_tags:
            if('\n' in i.text):
                price.append(i.text.split('\n')[1].replace('₹','').replace(',',''))
            else:
                price.append(i.text.replace('₹','').replace(',',''))
                        
    except NoSuchElementException:#handling no such element exception
        price.append('No details available')

In [88]:
# Fetching url to open each product.
flight_details_url = []
for _ in range(20):
    driver.execute_script("window.scrollBy(0,300)")
    time.sleep(1)
    
url = driver.find_elements_by_xpath('//span[@class="c-pointer c-secondary-500 hover:td-underline fs-2"]')
for i in url:
    try:
        flight_details_url.append(i.get_attribute('href'))
    except NoSuchElementException:
        flight_details_url.append('-')

In [89]:
len(flight_details_url)

44

In [91]:
# Scrap details for date of journey, source, destination.
date_of_journey = []
source = []
destination = []

for i in flight_details_url:
    #time.sleep(1) 
    
    #scarping date of journey
    try:                    
        doj_tags=driver.find_element_by_xpath("//div[@class='br-4 bw-1 bc-neutral-100 o-hidden ba mb-2']/div[2]/div[1]/div/div[2]/div[2]/span")
        date_of_journey.append(doj_tags.text.split(',')[1].strip().replace('Nov','11').replace(' ','/'))
    except NoSuchElementException:#handling no such element exception
        date_of_journey.append('No details available')
    
    #scarping flight source
    try:                    
        source_tags=driver.find_element_by_xpath("//h3[@class='fs-3 c-neutral-900 fw-500']")
        source.append(source_tags.text.split('→')[0])
    except NoSuchElementException:#handling no such element exception
        source.append('No details available') 
    
    #scarping flight destination
    try:                    
        destination_tags=driver.find_element_by_xpath("//h3[@class='fs-3 c-neutral-900 fw-500']")
        destination.append(destination_tags.text.split('→')[1])
    except NoSuchElementException:#handling no such element exception
        destination.append('No details available')

In [92]:
print(len(airline_name),len(date_of_journey),len(source),len(destination),len(departure_time),len(arrival_time),len(duration),len(flight_stop),len(price))

44 44 44 44 44 44 44 44 44


In [93]:
#Create dataframe.
import pandas as pd
df5 = pd.DataFrame({
    "Airline_name" : airline_name,
    "Date_of_journey" : date_of_journey,
    "Source" : source,
    "Destination" : destination,
    "Departure_time" : departure_time,
    "Arrival_time" : arrival_time,
    "Duration" : duration,
    "Total_stops" : flight_stop,
    "Price" : price,
})
df5

,Airline_name,Date_of_journey,Source,Destination,Departure_time,Arrival_time,Duration,Total_stops,Price
0,GO FIRST,10/11/2022,Pune,Bangalore,19:00,20:30,1h 30m,non-stop,5417
1,GO FIRST,10/11/2022,Pune,Bangalore,01:25,03:00,1h 35m,non-stop,5417
2,GO FIRST,10/11/2022,Pune,Bangalore,22:45,00:35,1h 50m,non-stop,5417
3,IndiGo,10/11/2022,Pune,Bangalore,01:00,02:35,1h 35m,non-stop,5418
4,IndiGo,10/11/2022,Pune,Bangalore,04:50,06:25,1h 35m,non-stop,5418
5,IndiGo,10/11/2022,Pune,Bangalore,07:25,09:00,1h 35m,non-stop,5418
6,IndiGo,10/11/2022,Pune,Bangalore,13:55,15:30,1h 35m,non-stop,5418
7,IndiGo,10/11/2022,Pune,Bangalore,19:15,20:50,1h 35m,non-stop,5418
8,IndiGo,10/11/2022,Pune,Bangalore,22:20,23:55,1h 35m,non-stop,5418
9,Air Asia,10/11/2022,Pune,Bangalore,23:55,01:20,1h 25m,non-stop,5419


In [94]:
# Concat all df into df_final
df_final = pd.concat([df,df2,df3,df4,df5],axis=0,ignore_index=True)
df_final

,Airline_name,Date_of_journey,Source,Destination,Departure_time,Arrival_time,Duration,Total_stops,Price
0,Air Asia,11/08/2022,New Delhi,Chennai,12:40,22:00,9h 20m,1 stop,7949
1,Vistara,11/08/2022,New Delhi,Chennai,13:35,16:25,2h 50m,non-stop,9418
2,Vistara,11/08/2022,New Delhi,Chennai,19:55,22:45,2h 50m,non-stop,9418
3,IndiGo,11/08/2022,New Delhi,Chennai,07:10,10:00,2h 50m,non-stop,9419
4,IndiGo,11/08/2022,New Delhi,Chennai,08:30,11:20,2h 50m,non-stop,9419
...,...,...,...,...,...,...,...,...,...
332,Air India,10/11/2022,Pune,Bangalore,07:30,16:15,8h 45m,1 stop,13293
333,Air India,10/11/2022,Pune,Bangalore,07:30,20:15,12h 45m,1 stop,13293
334,Air India,10/11/2022,Pune,Bangalore,18:50,08:55,14h 5m,1 stop,13293
335,Air India,10/11/2022,Pune,Bangalore,12:40,20:15,7h 35m,2 stops,13818


In [97]:
# Shuffle df_final 
df_final = df_final.sample(frac=1).reset_index(drop=True)
df_final

,Airline_name,Date_of_journey,Source,Destination,Departure_time,Arrival_time,Duration,Total_stops,Price
0,Air India,7/09/2022,Mumbai,Bangalore,19:00,08:55,13h 55m,1 stop,10070
1,Vistara,7/09/2022,Mumbai,Bangalore,15:45,09:00,17h 15m,1 stop,9280
2,Air India,11/08/2022,New Delhi,Chennai,16:15,19:30,3h 15m,non-stop,14880
3,SpiceJet,11/08/2022,New Delhi,Chennai,06:30,18:25,11h 55m,1 stop,18856
4,GO FIRST,10/11/2022,Pune,Bangalore,22:45,00:35,1h 50m,non-stop,5417
...,...,...,...,...,...,...,...,...,...
332,Air India,7/09/2022,Mumbai,Bangalore,07:20,00:05,16h 45m,2 stops,10915
333,Vistara,11/08/2022,New Delhi,Chennai,19:00,08:50,13h 50m,1 stop,16766
334,IndiGo,2/10/2022,Ahmedabad,Hyderabad,15:00,20:35,5h 35m,1 stop,11309
335,Air India,7/09/2022,Mumbai,Bangalore,19:00,20:15,25h 15m,1 stop,10070


In [146]:
df_final = pd.read_excel(r'flight_data_final.xlsx')
df_final

,Airline_name,Date_of_Journey,Source,Destination,Departure_time,Arrival_time,Duration,Total_stops,Price
0,Jet Airways,9/05/2022,Kolkata,Banglore,09:35,10:55:00,25h 20m,1 stop,10953
1,SpiceJet,3/04/2022,Kolkata,Banglore,17:10,19:40:00,2h 30m,non-stop,3841
2,IndiGo,1/06/2022,Delhi,Cochin,05:10,10:05:00,4h 55m,1 stop,6842
3,Jet Airways,27/05/2022,Delhi,Cochin,09:00,12:35:00,27h 35m,1 stop,12898
4,Vistara,7/09/2022,Mumbai,Bangalore,17:35,09:00,15h 25m,1 stop,9280
...,...,...,...,...,...,...,...,...,...
1530,SpiceJet,21/03/2022,Banglore,New Delhi,10:20,18:15:00,7h 55m,1 stop,7139
1531,Jet Airways,27/04/2022,Banglore,Delhi,06:00,08:45:00,2h 45m,non-stop,4544
1532,IndiGo,10/11/2022,Pune,Bangalore,22:20,23:55,1h 35m,non-stop,5418
1533,IndiGo,18/05/2022,Delhi,Cochin,08:35,16:10:00,7h 35m,1 stop,6442


In [144]:
# Save the file into excel.
df_final.to_excel(r'flight_data_final.xlsx',index=False)